In [1]:
import tensorflow as tf
import keras
from keras import backend as K
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
print(tf.__version__)
print(keras.__version__)
import cv2  # for image processing
from sklearn.model_selection import train_test_split
import scipy.io
import os
import h5py
from arts_preprocess_utils import load_dataset
from keras.preprocessing.image import ImageDataGenerator
import scipy
#from scipy.spatial.distance import directed_hausdorff, cosine
from IPython import display

Using TensorFlow backend.


1.2.1
2.0.6


In [2]:
# !!! remember to clear session/graph if you rebuild your graph to avoid out-of-memory errors !!!
def reset_tf_session():
    K.clear_session()
    tf.reset_default_graph()
    s = K.get_session()
    return s

In [3]:
reset_tf_session()

## Load data

**Metada**

In [4]:
df_artworks = pd.read_csv('/root/work/datasets/train_mayors_style.csv')
df_artworks.head()

,filename,title,style,genre,date,artist
0,70944.jpg,Forbidden Land,Surrealism,landscape,1937.0,Wolfgang Paalen
1,99442.jpg,Storm at sea,Romanticism,marina,1873,Ivan Aivazovsky
2,28908.jpg,Yachting in the Mediterranean,Realism,genre painting,1896,Julius LeBlanc Stewart
3,7486.jpg,Death of Eleazer,Romanticism,religious painting,1866,Gustave Dore
4,35766.jpg,The-Deluge,Romanticism,religious painting,NaN,Gustave Dore


**Image shape**

In [5]:
#According to the dataset input used during autoencoder training

img_Height = 128
img_Width = 128
chanels= 3
N_CLASSES = 1

### Build encoder

In [6]:
from models import Generator_model_complex, Discriminator_model

def build_deep_autoencoder(img_shape, code_size, filters = 40):
    
    encoder_model = Discriminator_model(filters=filters, code_shape=code_size, include_top=False)
    encoder = encoder_model.get_model(img_shape, N_CLASSES, False)
    
    decoder_model = Generator_model_complex(filters=filters, code_shape= (1,1,code_size))
    decoder = decoder_model.get_model(img_shape)
    
    return encoder, decoder

In [7]:
encoder, decoder = build_deep_autoencoder((img_Height, img_Width, 3), 300)

In [8]:
encoder.load_weights('./weigths/denoisy_encoder01.h5')

In [9]:
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 40)        1960      
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 40)        160       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64, 64, 40)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 80)        51280     
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 80)        320       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 32, 32, 80)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 160)       204960    
__________

## Encode artworks

In [10]:
import zipfile

#dataset from Kaggle
filezip = zipfile.ZipFile('/root/work/datasets/train.zip', 'r')

In [11]:
def get_image(path):
    #load image
    image = cv2.imread(path)
    image = cv2.resize(image, (img_Width, img_Height), interpolation=cv2.INTER_CUBIC)
    #normalize image
    image_norm = image * (1./255)
    image_norm = np.expand_dims(image_norm, axis=0)
    
    return image_norm
    

In [12]:
import os

def encode_artworks(x, encoder):

    try:
        img_path = filezip.extract('train/' + x)
        # Images
        image = get_image(img_path)

        # Encode
        code = encoder.predict(image)
        
        try:
            os.remove(img_path)
        except OSError as e:  ## if failed, report it back to the user ##
            print ("Error: %s - %s." % (e.filename, e.strerror))
            
        return code.reshape((-1,))

    except:
        print("Problem with file " + x)
        return None


In [13]:
df_artworks['code'] = df_artworks['filename'].apply(encode_artworks, encoder=encoder)

Problem with file 69008.jpg
Problem with file 121.jpg
Problem with file 38324.jpg
Problem with file 32150.jpg
Problem with file 32577.jpg
Problem with file 32192.jpg
Problem with file 50789.jpg


In [14]:
df_artworks.head()

,filename,title,style,genre,date,artist,code
0,70944.jpg,Forbidden Land,Surrealism,landscape,1937.0,Wolfgang Paalen,"[1.16539, 1.28992, 1.7879, 0.264377, 1.30938, ..."
1,99442.jpg,Storm at sea,Romanticism,marina,1873,Ivan Aivazovsky,"[0.0501826, 0.68001, 1.09941, 0.763883, 0.5918..."
2,28908.jpg,Yachting in the Mediterranean,Realism,genre painting,1896,Julius LeBlanc Stewart,"[-0.034542, 0.970973, 1.31414, 1.11773, 0.5739..."
3,7486.jpg,Death of Eleazer,Romanticism,religious painting,1866,Gustave Dore,"[0.387026, 0.229165, 0.831276, 0.408633, 0.812..."
4,35766.jpg,The-Deluge,Romanticism,religious painting,NaN,Gustave Dore,"[0.0899346, -0.0333367, 0.118036, 0.869628, 0...."


In [35]:
df_artworks['code'].isnull().value_counts()

False    46010
True         7
Name: code, dtype: int64

In [36]:
#Drop null values
df_artworks = df_artworks.dropna(subset=['code'])
df_artworks.shape

(46010, 7)

## Save values

In [37]:
df_artworks.to_csv('/root/work/datasets/train_mayors_style_encoded.csv', index=False)

In [38]:
encodes_list = list(df_artworks['code'])

In [40]:
encode_matrix = np.stack(encodes_list)
encode_matrix.shape

(46010, 300)

In [42]:
np.save('/root/work/datasets/train_mayors_style_encode', encode_matrix)

In [ ]:
df_toy = df_artworks[0:10]
df_toy

In [ ]:
df_toy['code'] = df_toy['filename'].apply(encode_artworks, encoder=encoder)

In [ ]:
df_toy